IMPLEMENT THE FUNCTIONS

In [51]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


Load the Dataset

In [52]:
df = pd.read_csv('/Users/elvisechefu/Desktop/language detection/languages.csv')

labels = df['language'].values
df.head(5)

,text,language
0,ich denke es handelt sich hier um ein missvers...,german
1,ich habe tom gerade erst verlassen,german
2,tom versuchte mary nur zu ärgern,german
3,tom hat mir die hand geküsst,german
4,ich wusste dass dir das gefiele,german


PROCESS THE TEXT : LOWERCASING, TOKENIZATION, REMOVAL OF PUNCTUATION< SPECAIL CHARACTERS AND STOPWORDS

In [53]:
# Lowercasing
df['clean_text'] = df['text'].apply(lambda x: x.lower())

# Tokenization
df['tokenized_text'] = df['clean_text'].apply(nltk.word_tokenize)

# Removing Punctuation and Special Characters
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

df['clean_text'] = df['clean_text'].apply(remove_special_characters)

# Removing Stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df['preprocessed_text'] = df['tokenized_text'].apply(remove_stopwords)

Define a function to extract character-level features:

In [54]:
def extract_char_features(document, word_length=4):
    document = re.sub(r'[^a-zA-Z0-9\s]', '', document)
    features = {}
    for i in range(1, word_length+1):
        features['char-%s' % i] = [0]*26
        for j in range(len(document)-i+1):
            try:
                features['char-%s' % i][ord(document[j]) - ord('a')] += 1
            except:
                features['char-%s' % i][ord(document[j]) - ord('A')] += 1
    return features

EXTRACT CHARACTER-LEVEL FEATURES FOR EACH DOC

In [55]:
X = []
for index, row in df.iterrows():
    if row['preprocessed_text']:  # Check if the list is not empty
        X.append(extract_char_features(row['preprocessed_text'][0]))
    else:
        # Handle the case where the list is empty
        # For example, you could append a placeholder value or skip this row
        pass
X = np.array(X)


SPLIT THE DATASET INTO TEST VALIDATE AND TRAIN SET
Whilst trying to split the data i found some duplicates in the labels, so i removed the label.

In [56]:
#Remove extra labels
aligned_labels = labels[:len(X)]

labels = df[df['preprocessed_text'].apply(bool)]['language'].values

X_train_val, X_test, y_train_val, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

In [63]:
# Convert list of dictionaries to 2D numpy array
X_train_array = np.array([np.array(list(d.values())) for d in X_train])

SVM_CLF is SVM classifier 

In [64]:
# Create an SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)

SVM MODEL

In [65]:
# Create an SVM model
svm_model = SVC(kernel='linear', random_state=42)

In [69]:
import numpy as np

# Assuming X_train is a list of dictionaries
# Flatten values from each dictionary and stack them into an array
X_train_array = np.array([np.array(list(d.values())).flatten() for d in X_train])

# Fit the classifier to the training data
svm_clf.fit(X_train_array, y_train)